### 세 파트 모두 연동한 코드

In [1]:
import os
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 모델 경로
model_paths = {
    "ei": ("models/ei_bilstm_model2.h5", "tokenizers/ei_tokenizer2.pkl"),
    "ns": ("models/ns_bilstm_model2.h5", "tokenizers/ns_tokenizer2.pkl"),
    "tf": ("models/tf_bilstm_model.h5", "tokenizers/tf_tokenizer.pkl"),
    "jp": ("models/jp_bilstm_model.h5", "tokenizers/jp_tokenizer.pkl"),
}

models, tokenizers = {}, {}
max_len = 300

for trait, (model_file, tokenizer_file) in model_paths.items():
    models[trait] = tf.keras.models.load_model(model_file)
    with open(tokenizer_file, "rb") as f:
        tokenizers[trait] = pickle.load(f)


In [2]:
def predict_mbti_per_trait_threshold(text, models, tokenizers, thresholds, max_len=300):
    label_pairs = {"ei": ("E", "I"), "ns": ("N", "S"), "tf": ("T", "F"), "jp": ("J", "P")}
    result = ""
    debug_log = []

    for trait in ["ei", "ns", "tf", "jp"]:
        tokenizer = tokenizers[trait]
        model = models[trait]
        seq = tokenizer.texts_to_sequences([text])
        padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_len, padding='post')
        pred = model.predict(padded)[0][0]
        threshold = thresholds.get(trait, 0.5)

        
        label = label_pairs[trait][0] if pred >= threshold else label_pairs[trait][1]

        debug_log.append((trait.upper(), pred, label))
        result += label

    return result, debug_log


In [12]:
# 대표 질문 답변 (각 MBTI 지표별 1개)
session_inputs = [
    "새로운 아이디어를 떠올리는 걸 정말 좋아해요.",
    "사람들과 토론하면서 생각을 넓히는 게 즐거워요.",
    "계획대로 움직이기보다는 그때그때 흥미로운 걸 따라가요.",
    "틀에 박힌 방식보다는 창의적인 해결책을 선호해요.",
    "상대방의 논리를 반박하면서 더 나은 방향을 찾는 걸 즐겨요.",
    "여러 가지 가능성을 동시에 고려하는 편이에요.",
    "새로운 프로젝트나 시도를 두려워하지 않아요.",
    "혼자보다 여러 사람과 함께할 때 더 에너지가 나요.",
    "감정보다 논리와 전략으로 접근하는 걸 선호해요.",
    "즉흥적인 결정이 오히려 더 좋은 결과를 가져올 때도 많다고 생각해요."
]

combined_text = " ".join(session_inputs)


# 예측 실행 (지표별 threshold 적용)
thresholds = {"ei": 0.40, "ns": 0.85, "tf": 0.62, "jp": 0.50}
predicted_mbti, logs = predict_mbti_per_trait_threshold(combined_text, models, tokenizers, thresholds)

# 결과 출력
print("\n[예측 상세 로그]")
for trait, prob, label in logs:
    print(f"{trait}: 확률={prob:.4f} → 분류결과={label}")
print(f"\n[최종 예측 MBTI]: {predicted_mbti}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

[예측 상세 로그]
EI: 확률=0.4005 → 분류결과=E
NS: 확률=0.8752 → 분류결과=N
TF: 확률=0.6519 → 분류결과=T
JP: 확률=0.4349 → 분류결과=P

[최종 예측 MBTI]: ENTP
